# RASTER BLASTER

## NCRP Raster to Point CSV Tool

This notebook is designed to accept a geotiff file, convert the raster to point geometry, standardize values, add additional fields, convert geometry to lat/long coordinates, and then export as a CSV.

Please update your workspace, input raster, and output point file to your directory and desired files.

In [26]:
import pandas as pd
import arcpy
from arcpy import env
import arcgis

# Change env.workspace to "C:/your project directory"
env.workspace = "C:/Users/Kylew/Dropbox/Capstone/Data/Tree_Species_Density/Sitka_Spruce_Automation"

In [28]:
# Convert raster to point data

# Input raster - change to your desired raster
inRaster = "WA_sitka_spruce_stand_density.tif"
# Output raster to point file - change to your desired point file name
outPoint = "sitka_spruce_point.shp"
# Field
field = "VALUE"

arcpy.RasterToPoint_conversion(inRaster, outPoint, field)

In [30]:
# Standardize grid_code to minimum/maximum normalized value

# Input layer for standardization
inputFeature = outPoint[:-4]

# Set the field(s) that will be standardized
field = "grid_code norm_value"

# Set the standardization method
method = "MIN-MAX"

arcpy.management.StandardizeField(inputFeature, field, method)

<Result 'sitka_spruce_point'>

In [32]:
# Add lat and long fields to table

# inputFeature
field1 = "long"
field2 = "lat"
field_type = "DOUBLE"

arcpy.AddField_management(inputFeature, field1, field_type)
arcpy.AddField_management(inputFeature, field2, field_type)

<Result 'sitka_spruce_point'>

In [35]:
# Calculate Geometry

# inputFeature
# Coordinate System

coord_system = arcpy.SpatialReference("USA Contiguous Albers Equal Area Conic USGS")
# Coordinate format = decimal degrees
coord_format = "DD"

arcpy.CalculateGeometryAttributes_management(inputFeature,
    [["long", "POINT_X"], 
     ["lat", "POINT_Y"]],
    coordinate_system = coord_system,
    coordinate_format = coord_format)

<Result 'sitka_spruce_point'>

In [37]:
# Convert table to CSV

# inputFeature = your input value defined earlier
df = pd.DataFrame(arcpy.da.FeatureClassToNumPyArray(in_table=inputFeature,
    field_names=["long", "lat", "norm_value"]))
output_name = "WA_" + inputFeature + "_stand_density.csv"
df.to_csv(output_name, index=False)